In [ ]:
!pip3 install mysql-connector-python

In [ ]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
import getpass
import nba_api as nba
import tqdm
#Function citations https://www.freecodecamp.org/news/connect-python-with-sql/
#https://github.com/swar/nba_api/blob/master/docs/nba_api/stats/endpoints/playergamelog.md

In [ ]:
def create_server_connection(host_name, user_name, user_password):
    """
    Connect to mySql Server
    """
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection


connection = create_server_connection("localhost", "root")

In [ ]:
def create_database(connection, query):
    """
    From: https://www.freecodecamp.org/news/connect-python-with-sql/
    """
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as err:
        print(f"Error: '{err}'")
        
        
create_database_query = "CREATE DATABASE PickAndRoll"
create_database(connection, create_database_query)

In [ ]:
def create_db_connection(host_name, user_name, db_name):
    """
    From: https://www.freecodecamp.org/news/connect-python-with-sql/
    """
    user_password = getpass.getpass("Password")
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection
connection = create_db_connection("localhost", "root", "PickAndRoll")

In [ ]:
def execute_query(connection, query):
    """
    From: https://www.freecodecamp.org/news/connect-python-with-sql/
    """
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        # print("Query successful")
        # return record
    except Error as err:
        print(f"Error: '{err}'")
        
drop_table_organization = """DROP TABLE Organization;"""
execute_query(connection, drop_table_organization)        
create_table_organization = """CREATE TABLE Organization (
    OrgId INT,
    OrgAbbv VARCHAR(5),
    OrgName VARCHAR(255),
    PRIMARY KEY(OrgId)
);
"""
print(create_table_organization)
execute_query(connection, create_table_organization)

In [ ]:
drop_table_team = """DROP TABLE Team;"""
execute_query(connection, drop_table_team)
create_table_team = """CREATE TABLE Team (
    TeamId INT,
    Year INT,
    OrgId INT,
    PRIMARY KEY(TeamId),
    FOREIGN KEY (OrgId) REFERENCES Organization(OrgId) ON DELETE CASCADE
);
"""
execute_query(connection, create_table_team)

In [ ]:
drop_table_player = """DROP TABLE Player;"""
execute_query(connection, drop_table_player)
create_table_player = """CREATE TABLE Player (
    PlayerId INT,
    FirstName VARCHAR(255),
    LastName VARCHAR(255),
    Height VARCHAR(255),
    Weight INT,
    PRIMARY KEY(PlayerId)
);
"""
execute_query(connection, create_table_player)

In [ ]:
drop_table_game = """DROP TABLE Game;"""
execute_query(connection, drop_table_game)
create_table_game = """CREATE TABLE Game (
    GameId INT,
    Date Date,
    HomeScore INT,
    AwayScore INT,
    HomeTeamId INT,
    AwayTeamId INT,
    PRIMARY KEY(GameId),
    FOREIGN KEY(HomeTeamId) REFERENCES Team(TeamId) ON DELETE CASCADE,
    FOREIGN KEY(AwayTeamId) REFERENCES Team(TeamId) ON DELETE CASCADE
);
"""
execute_query(connection, create_table_game)

In [ ]:
# drop_table_box_score = """DROP TABLE BoxScore;"""
# execute_query(connection, drop_table_box_score)
create_table_box_score = """CREATE TABLE BoxScore (
   BoxScoreId INT PRIMARY KEY,
   Pts INT,
   Asts INT,
   Rebs INT,
   Blks INT,
   Stls INT,
   TOs INT,
   FGM INT,
   FGA INT,
   TPM INT,
   TPA INT,
   FTM INT,
   FTA INT,
   PlayerId INT,
   GameId INT,
   FOREIGN KEY(PlayerId) REFERENCES Player(PlayerId) ON DELETE CASCADE,
   FOREIGN KEY(GameId) REFERENCES Game(GameId) ON DELETE CASCADE
);
"""
execute_query(connection, create_table_box_score)

In [ ]:
drop_table_sign = """DROP TABLE Sign;"""
execute_query(connection, drop_table_sign)
create_table_sign = """CREATE TABLE Sign (
   TeamId INT,
   PlayerId INT,
   Year INT,
   PRIMARY KEY (TeamId, PlayerId),
   FOREIGN KEY(TeamId) REFERENCES Team(TeamId) ON DELETE CASCADE,
   FOREIGN KEY(PlayerId) REFERENCES Player(PlayerId) ON DELETE CASCADE
);
"""
execute_query(connection, create_table_sign)

In [ ]:
# drop_table_player_play = """DROP TABLE PlayerPlay"""
# execute_query(connection, drop_table_player_play)
create_table_player_play = """CREATE TABLE PlayerPlay (
    GameId INT,
    PlayerId INT,
    PRIMARY KEY (GameId, PlayerId),   
    FOREIGN KEY(GameId) REFERENCES Game(GameId) ON DELETE CASCADE,
    FOREIGN KEY(PlayerId) REFERENCES Player(PlayerId) ON DELETE CASCADE
)
"""
execute_query(connection, create_table_player_play)

In [ ]:
drop_table_guesser_instance = """DROP TABLE GuesserInstance"""
execute_query(connection, drop_table_guesser_instance)
create_table_guesser_instance = """CREATE TABLE GuesserInstance (
    GameId INT PRIMARY KEY,
    NumGuesses INT,
    Name VARCHAR(255), 
    GaveUp BOOLEAN,
    UserId INT,
    FOREIGN KEY(UserId) REFERENCES User(UserId) ON DELETE CASCADE
)
"""
execute_query(connection, create_table_guesser_instance)

In [ ]:
drop_table_user = """DROP TABLE User"""
execute_query(connection, drop_table_user)
create_table_user = """CREATE TABLE User (
    UserId INT PRIMARY KEY,
    Name VARCHAR(255)
)
"""
execute_query(connection, create_table_user)

In [ ]:
from nba_api.stats.static import teams
all_teams = teams.get_teams()
org_tuples = []
for team in all_teams:
    org_tuples.append((team['id'], team['abbreviation'], team['full_name']))

for tup in org_tuples:
    insert_org = f"""INSERT INTO Organization(OrgId, OrgAbbv, OrgName)
VALUES{tup};"""
    execute_query(connection, insert_org)



In [ ]:
from nba_api.stats.static import teams
all_teams = teams.get_teams()
teams_tuples = []
index = 1
for team in all_teams:
    for j in range(2):
        teams_tuples.append((index + j, f'202{j}',team['id']))
    index += 2
for tup in teams_tuples:
    insert_team = f"""INSERT INTO Team(TeamId, Year, OrgId)
VALUES{tup};"""
    execute_query(connection, insert_team)

In [ ]:
from nba_api.stats.endpoints.commonallplayers import CommonAllPlayers

all_players = CommonAllPlayers(league_id='00')
df = all_players.common_all_players.get_data_frame()
df.drop_duplicates().sort_values(by=['PERSON_ID'])
df['TO_YEAR'] = df['TO_YEAR'].astype('int')


recent_players_df = df[df['TO_YEAR'] >= 2020]
player_ids = recent_players_df['PERSON_ID'].values.tolist()

In [ ]:
from nba_api.stats.endpoints.commonplayerinfo import CommonPlayerInfo
from tqdm import tqdm
import pandas as pd

data = pd.DataFrame(columns=['PERSON_ID', 'FIRST_NAME', 'LAST_NAME', 'HEIGHT', 'WEIGHT'])

for i in tqdm(range(len(player_ids))):
    player = CommonPlayerInfo(player_id=player_ids[i]).common_player_info.get_data_frame()
    player = player[['PERSON_ID', 'FIRST_NAME', 'LAST_NAME', 'HEIGHT', 'WEIGHT']]
    data = pd.concat([data,player], axis=0)

In [ ]:
nba_api_to_guesser_2020 = {}
nba_api_to_guesser_2021 = {}
i = 1
for team in all_teams:
    nba_api_to_guesser_2020[team['id']] = i
    nba_api_to_guesser_2021[team['id']] = i + 1
    i += 2
print(nba_api_to_guesser_2020, nba_api_to_guesser_2021)

In [ ]:
df = pd.read_csv("players.csv", usecols=range(1,6))
for player in list(df.itertuples(index=False, name=None)):
    player_query = f"""INSERT INTO Player(PlayerId, FirstName, LastName, Height, Weight)
    VALUES{player}"""
    execute_query(connection, player_query)

In [ ]:
from nba_api.stats.endpoints.leaguegamelog import LeagueGameLog

all_games = LeagueGameLog(league_id='00', season="2020-21", season_type_all_star="Regular Season")
df = all_games.league_game_log.get_data_frame()
df.drop_duplicates().sort_values(by=['GAME_ID'])
# df['TO_YEAR'] = df['TO_YEAR'].astype('int')


# recent_players_df = df[df['TO_YEAR'] >= 2020]
# player_ids = recent_players_df['PERSON_ID'].values.tolist()
# print(df)
# df.to_csv("games.csv")




data = pd.DataFrame(columns=['GAME_ID', 'GAME_DATE', 'HOME_SCORE', 'AWAY_SCORE', 'HOME_TEAM_ID', 'AWAY_TEAM_ID'])
for i in tqdm(range(0, len(df), 2)):
    game = {'GAME_ID': df.loc[i]['GAME_ID'], 'GAME_DATE': df.loc[i]['GAME_DATE'], 'HOME_SCORE': df.loc[i]['PTS'], 'AWAY_SCORE': df.loc[i + 1]['PTS'], 'HOME_TEAM_ID': df.loc[i]['TEAM_ID'], 'AWAY_TEAM_ID': df.loc[i + 1]['TEAM_ID']}
    if 
    game = pd.Series(game)
    # print(game)
#     player = CommonPlayerInfo(player_id=player_ids[i]).common_player_info.get_data_frame()
#     player = player[['PERSON_ID', 'FIRST_NAME', 'LAST_NAME', 'HEIGHT', 'WEIGHT']]
    data = data.append(game, ignore_index = True)  
print(data)

In [ ]:
# 
# data.to_csv("games2020-2021.csv")
all_teams = teams.get_teams()
df = pd.read_csv("csv/games2021-2022.csv", usecols=range(1,7))
# print(df)
for game in list(df.itertuples(index=False, name=None)):
    game = list(game)
    game[-1] = nba_api_to_guesser_2021[game[-1]]
    game[-2] = nba_api_to_guesser_2021[game[-2]] 
    # print(game)

    game_query = f"""INSERT INTO Game(GameId, Date, HomeScore, AwayScore, HomeTeamId, AwayTeamId)
    VALUES{tuple(game)}"""
    execute_query(connection, game_query)

In [ ]:
from nba_api.stats.endpoints.commonteamroster import CommonTeamRoster
from nba_api.stats.static import teams
all_teams = teams.get_teams()
import time
# # print(all_teams)
df = pd.DataFrame(columns=['TeamID', 'SEASON', 'LeagueID', 'PLAYER', 'PLAYER_SLUG', 'NUM', 'POSITION', 'HEIGHT', 'WEIGHT', 'BIRTH_DATE', 'AGE', 'EXP', 'SCHOOL', 'PLAYER_ID'])
for l in tqdm(range(len(all_teams))):
    rosters = CommonTeamRoster(season = "2020", team_id= all_teams[l]['id'])
    df = pd.concat([df, rosters.common_team_roster.get_data_frame()], axis = 0)
    time.sleep(.600)

# df.to_csv("sign2020.csv")

In [ ]:

sign = pd.read_csv("sign2021.csv", usecols = range(1, 16))

sign.drop(['LeagueID', 'PLAYER', 'PLAYER_SLUG', 'NUM', 'POSITION', 'HEIGHT', 'WEIGHT', 'BIRTH_DATE', 'AGE', 'EXP', 'SCHOOL', 'NICKNAME'], axis = 1, inplace=True)
# print(sign)

for signings in list(sign.itertuples(index=False, name=None)):
    signings = list(signings)
    signings[0] = nba_api_to_guesser_2021[signings[0]]
    # print(signings)
    sign_query = f"""INSERT INTO Sign(TeamId, Year, PlayerId)
    VALUES{tuple(signings)}"""
    execute_query(connection, sign_query)

In [ ]:
query = """SELECT COUNT(*)
FROM (
    SELECT g1.GameId
    FROM Team t1 JOIN Game g1 ON(t1.TeamId = g1.HomeTeamId)
    WHERE (g1.HomeScore > g1.AwayScore AND g1.HomeTeamId = 1610612737)

    UNION

    SELECT g2.GameId
    FROM Team t2 JOIN Game g2 ON(t2.TeamId = g2.AwayTeamId)
    WHERE (g2.AwayScore > g2.HomeScore AND g2.AwayTeamId = 1610612737)
) AS winning_games;"""
print(query)
execute_query(connection, query)

In [ ]:
# %pip install numba
twenty_games = pd.read_csv("csv/games2021-2022.csv", usecols=range(1,7))
from tqdm import tqdm 
import time
# print(twenty_games)
import numpy as np
from numba import prange, njit

from nba_api.stats.endpoints import boxscoretraditionalv2


# print(twenty_games)
boxscore = []
# print(boxscore)
# p

# @njit(parallel=True, nopython=True)
# def insert_boxscore(entry, boxscore):
#     response = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id= "00" + str(entry))
#     df = response.player_stats.get_data_frame()
#     # print(df.keys())
#     boxscore = pd.concat([boxscore, df])
#     time.sleep(0.600)


# def func():
#     boxscore = pd.DataFrame(columns = ['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
#        'PLAYER_NAME', 'NICKNAME', 'START_POSITION', 'COMMENT', 'MIN', 'FGM',
#        'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
#        'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS',
#        'PLUS_MINUS'])
for game_id_index in tqdm(range(len(twenty_games['GAME_ID']))):
    response = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id= "00" + str(twenty_games['GAME_ID'][game_id_index]))
    df = response.player_stats.get_data_frame()
    # print(df.keys())
    boxscore.append(df)
    time.sleep(0.600)



# print(boxscore)
# print(df)
    # break

In [ ]:
print(len(boxscore))

boxscore_csv = boxscore[0]

for i in tqdm(range(1, len(boxscore))):
    boxscore_csv = pd.concat([boxscore_csv, boxscore[i]])


boxscore_csv.to_csv("boxscores_2021_2022.csv")




In [ ]:
connection = create_db_connection("localhost", "root", "PickAndRoll")

In [ ]:
# boxscore_csv = pd.read_csv("boxscores.csv", usecols=range(1))
# print(boxscore_csv.drop(['TEAM_ABBREVIATION', 'TEAM_CITY','PLAYER_NAME', 'NICKNAME', 'START_POSITION', 'COMMENT', 'MIN', 'FG_PCT', 'FG3_PCT', 'FT_PCT','OREB', 'DREB', 'TEAM_ID', 'PF','PLUS_MINUS'], axis = 1, inplace=True))
# boxscore_csv.fillna(0, inplace=True)
print(boxscore_csv)
# for value in boxscore_csv['Game_ID']
# boxscore_csv.drop('Unnamed: 0', axis = 1, inplace=True)

# print(connection.disconnect())
connection = create_db_connection("localhost", "root", "PickAndRoll")
i = 28860

boxscore_list = list(boxscore_csv.itertuples(index=False, name=None))
# print(boxscore_csv.keys())
for box in tqdm(range(len(boxscore_list))):
    boxscore_list[box] = list(boxscore_list[box])
    boxscore_list[box].append(i)
    for value in range(len(boxscore_list[box])):
        if type(boxscore_list[box][value]) == float:
           boxscore_list[box][value] = int(boxscore_list[box][value]) 
    boxscore_list[box] = tuple(boxscore_list[box])
    # print(boxscore_list[box])


    boxScoreQuery = f"""INSERT INTO BoxScore(GameId, PlayerId, FGM, FGA, TPM, TPA, FTM, FTA, Rebs, Asts, Stls, Blks, TOs, Pts, BoxScoreId)
    VALUES{boxscore_list[box]}"""
    execute_query(connection, boxScoreQuery)
    i+=1
#     # break

In [ ]:
import numpy as np
boxscore_csv.replace([None], [0], inplace=True)

# boxscore_csv = boxscore_csv.astype("int")

print(boxscore_csv)

In [ ]:
players_play = pd.read_csv("playerplay.csv", usecols=(1,2))
print(players_play)


pp = list(players_play.itertuples(index=False, name=None))
for index in tqdm(range(len(pp))):
    # print(pp[index])
    pp_query = f"""INSERT INTO PlayerPlay(PlayerId, GameId)
    VALUES{pp[index]};
    """
    execute_query(connection, pp_query)
    # break